In [151]:
# import Dependencies
import pandas as pd
import numpy as np

In [152]:
# Create path with a variable 
heroes_path = "../Resources/purchase_data.csv"

In [153]:
# Read in the csv from the path
heroes_df = pd.read_csv(heroes_path)
heroes_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [154]:
# Check to see if there is any missing data that needs to be removed
heroes_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [155]:
# look up sn's unique value
heroes_df["SN"].describe()

count           780
unique          576
top       Lisosia93
freq              5
Name: SN, dtype: object

In [156]:
# Create a new dataframe that is grouped by SN to remove any duplicate data for number of players
sn_grouped_df = heroes_df.groupby(["SN"])
print(sn_grouped_df)
sn_grouped_df.count().head()

,Purchase ID,Age,Gender,Item ID,Item Name,Price
SN,,,,,,
Adairialis76,1,1,1,1,1,1
Adastirin33,1,1,1,1,1,1
Aeda94,1,1,1,1,1,1
Aela59,1,1,1,1,1,1
Aelaria33,1,1,1,1,1,1


In [157]:
# Get total player amount with len() on SN
total_players = len(sn_grouped_df["SN"])
total_players

576

##  Player Count
 
    Display the total number of players

In [158]:
# Create dataframe for total players with sn_count variable as to value
total_players_df = pd.DataFrame({"Total Players": [total_players]})
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

- Run basic calculations to obtain number of unique items, average price, etc.


- Create a summary data frame to hold the results


- Optional: give the displayed data cleaner formatting


- Display the summary data frame

In [159]:
# Get unique items count with len() and unique() fucntion
unique_items_count = len(heroes_df["Item ID"].unique())

In [160]:
# Get average from Price column using the mean() function
average = heroes_df["Price"].mean()

In [161]:
# Get total number of purchases with len() function 
total_purchases = len(heroes_df)

In [162]:
# Get total revenue from Price column using sum() function
total_revenue = heroes_df["Price"].sum()

In [215]:
# Create dataframe using a dictionary with keys (headers) and values collected above
purchasing_analysis_df = pd.DataFrame({"Number of Unique Items": [unique_items_count], "Average Price": [average], 
                                       "Number of Purchases": [total_purchases], "Total Revenue": [total_revenue] })

# .map to format currency
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].map("${:,.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${:,.2f}".format)
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


 ## Gender Demographics
 
    
    Percentage and Count of Male Players


    Percentage and Count of Female Players


    Percentage and Count of Other / Non-Disclosed

In [216]:
# .groupby gender and assure to use SN.nunique() so that it only counts user once 
group_by_gender = heroes_df.groupby("Gender")

# Calcuate values from group_by_gender_df that will be used to create gender_table dataframe  
# (nunique is used to convert a string into a # value)
gender_count = group_by_gender["SN"].nunique()
gender_percentage = (gender_count / total_players) * 100

# Create dataframe for gender demographics table with above variables
gender_table_df = pd.DataFrame({"Total Count": gender_count, "Percentage of Players": gender_percentage})

# sort table to by total count, highest on top
gender_table_sort_df = gender_table_df.sort_values("Total Count", ascending=False)
gender_table_sort_df

# .map to format percentage
gender_table_sort_df["Percentage of Players"] = gender_table_sort_df["Percentage of Players"].map("{:.2f}%".format)
gender_table_sort_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


 ## Purchasing Analysis (Gender)
 
    Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender


    Create a summary data frame to hold the results


    Optional: give the displayed data cleaner formatting


    Display the summary data frame

In [231]:
# Calcuate values from group_by_gender_df that will be used to create gender_demographics_df  
# (nunique is used to convert a string into a # value)
gender_purchase_count = group_by_gender["Price"].count()
total_value_per_gender = group_by_gender["Price"].sum()
average_gender_purchase_price = (total_value_per_gender / gender_purchase_count)
average_per_person = (total_value_per_gender / gender_count)

# Create gender_demographics dataframe with values created above. 
gender_demographics_df = pd.DataFrame({"Purchase Count": gender_purchase_count, "Average Purchase Price": average_gender_purchase_price, 
                                      "Total Purchase Value": total_value_per_gender, 
                                       "Avg Total Purchase per Person": average_per_person})

# .map to format "Average Purchase Price", "Total Purchase Value", and "Avg Total Purchasee per Person"
gender_demographics_df["Average Purchase Price"] = gender_demographics_df["Average Purchase Price"].map("${:.2f}".format)
gender_demographics_df["Total Purchase Value"] = gender_demographics_df["Total Purchase Value"].map("${:.2f}".format)
gender_demographics_df["Avg Total Purchase per Person"] = gender_demographics_df["Avg Total Purchase per Person"].map("${:.2f}".format)

gender_demographics_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics
 
    Establish bins for ages


    Categorize the existing players using the age bins. Hint: use pd.cut()


    Calculate the numbers and percentages by age group


    Create a summary data frame to hold the results


    Optional: round the percentage column to two decimal points


    Display Age Demographics Table

In [169]:
# Use .loc to pull column needed and use .drop_duplicates to remove duplicate SN
unique_age_df = heroes_df.loc[:, ["SN", "Age", "Gender"]]
unique_age_df = unique_age_df.drop_duplicates() 

# use count to confirm the amount matches total players number 
unique_age_df.count()

SN        576
Age       576
Gender    576
dtype: int64

In [170]:
# Establish bins for ages
# reminder - bin is up to so that include whole numbers so going .01 under.
age_bin = [0, 9.99, 14.99, 19.99, 24.99, 29.99 ,34.99, 39.99, 99.99]

# reminder - lables area range for the bin data, so one less value  
bin_name = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [171]:
# Binning! bin df with "new header" using - pd.cut df "old header", apply bin and label from above, lowest=True
unique_age_df["Age Range"] = pd.cut(unique_age_df["Age"], age_bin, labels=bin_name, include_lowest=True )

In [218]:
# .groupby "Age Range"
heroes_age_group = unique_age_df.groupby("Age Range")

# get calulations using heroes_age_group for age demopraghics 
age_count = heroes_age_group["Age Range"].count()
age_percentage_of_players = (age_count / total_players) *100

# Create dataframe with variables above
age_demographics_df = pd.DataFrame({"Total Count": age_count, "Percentage of Players": age_percentage_of_players})

# .map to format "percentage of Players"
age_demographics_df["Percentage of Players"] = age_demographics_df["Percentage of Players"].map("{:.2f}%".format)

age_demographics_df

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)
 
    Bin the purchase_data data frame by age


    Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table    below


    Create a summary data frame to hold the results


    Optional: give the displayed data cleaner formatting


    Display the summary data frame

In [186]:
# .loc to get columns needed for code
purchase_age_df = heroes_df.loc[:, ["SN", "Age", "Purchase ID", "Price"]]
purchase_age_df = purchase_age_df.drop_duplicates()  
purchase_age_df.count()

SN             780
Age            780
Purchase ID    780
Price          780
dtype: int64

In [187]:
# Get max age
heroes_df["Age"].max()

45

In [188]:
# Establish bins for ages
# reminder - bin is up to so that include whole numbers so going .01 under.
age_bin = [0, 9.99, 14.99, 19.99, 24.99, 29.99 ,34.99, 39.99, 99.99]

# reminder - lables area range for the bin data, so one less value  
bin_name = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [189]:
# Binning! bin df with "new header" using - pd.cut df "old header", apply bin and label from above, lowest=True
purchase_age_df["Age Range"] = pd.cut(purchase_age_df["Age"], age_bin, labels=bin_name, include_lowest=True )

In [219]:
# .groupby "Age Range"
purchase_group_df = purchase_age_df.groupby("Age Range")

# calculations from purchase_group_df for age_purchase_df
age_purchase_count = purchase_group_df["Purchase ID"].count()
age_total_purchase = purchase_group_df["Price"].sum()
age_avg_purchase_price = (age_total_purchase / age_purchase_count)
age_avg_total_per_person = (age_total_purchase / age_count )


# Create df from vairables above
age_purchase_df = pd.DataFrame({"Purchase Count": age_purchase_count, "Average Purchase Price": age_avg_purchase_price, 
                                "Total Purchase Value": age_total_purchase, "Avg Total Purchase per Person": age_avg_total_per_person})

age_purchase_df["Average Purchase Price"] = age_purchase_df["Average Purchase Price"].map("${:.2f}".format)
age_purchase_df["Total Purchase Value"] = age_purchase_df["Total Purchase Value"].map("${:.2f}".format)
age_purchase_df["Avg Total Purchase per Person"] = age_purchase_df["Avg Total Purchase per Person"].map("${:.2f}".format)

age_purchase_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders
 
    Run basic calculations to obtain the results in the table below

    
    Create a summary data frame to hold the results


    Sort the total purchase value column in descending order


    Optional: give the displayed data cleaner formatting


    Display a preview of the summary data frame

In [198]:
# Use sn_grouped_df as it was already group by  SN earlier in the code 
# Using count() for next code
sn_grouped_df.count().head()

,Purchase ID,Age,Gender,Item ID,Item Name,Price
SN,,,,,,
Adairialis76,1,1,1,1,1,1
Adastirin33,1,1,1,1,1,1
Aeda94,1,1,1,1,1,1
Aela59,1,1,1,1,1,1
Aelaria33,1,1,1,1,1,1


In [220]:
# Create calulations from sn_grouped_df for top_spenders_df
sn_purchase_count = sn_grouped_df["Purchase ID"].count()
sn_price = sn_grouped_df["Price"].count()
sn_total_purchase_value = sn_grouped_df["Price"].sum()
sn_average_purchase_price = (sn_total_purchase_value / sn_purchase_count)


# Create df from variables above
sn_table_df = pd.DataFrame({"Purchase Count": sn_purchase_count, "Average Purchase Price": sn_average_purchase_price,
                         "Total Purchase Value": sn_total_purchase_value})

# Create sorted df by "Purchase Count", highest on top
top_spenders_df = sn_table_df.sort_values("Purchase Count", ascending=False)

# .map to format "Average Purchase Price" and "Total Purchase Value"
top_spenders_df["Average Purchase Price"] = top_spenders_df["Average Purchase Price"].map("${:.2f}".format)
top_spenders_df["Total Purchase Value"] = top_spenders_df["Total Purchase Value"].map("${:.2f}".format)

top_spenders_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


## Most Popular Items
 
    Retrieve the Item ID, Item Name, and Item Price columns

    
    Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


    Create a summary data frame to hold the results


    Sort the purchase count column in descending order


    Optional: give the displayed data cleaner formatting


    Display a preview of the summary data frame

In [228]:
# .groupby("Items ID" and "Item Name")
grouped_items_df = heroes_df.groupby(["Item ID", "Item Name"])

# Get calulations for from group_items_df
item_purchase_count = grouped_items_df["Item ID"].count()
item_total_purchase_value = grouped_items_df["Price"].sum()
item_price = (item_total_purchase_value / item_purchase_count)

# Create df with variables above
item_table_df = pd.DataFrame({"Purchase Count": item_purchase_count, "Item Price": item_price, 
                              "Total Purchase Value": item_total_purchase_value})

# .map to format "Item Price" and "Total Purchase Value"
item_table_df["Item Price"] = item_table_df["Item Price"].map("${:.2f}".format)
item_table_df["Total Purchase Value"] = item_table_df["Total Purchase Value"].map("${:.2f}".format)

item_table_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
0,Splinter,4,$1.28,$5.12
1,Crucifer,4,$2.94,$11.77
2,Verdict,6,$2.48,$14.88
3,Phantomlight,6,$2.49,$14.94
4,Bloodlord's Fetish,5,$1.70,$8.50


## Most Profitable Items
 
    Sort the above table by total purchase value in descending order


    Optional: give the displayed data cleaner formatting


    Display a preview of the data frame

In [229]:
# sorting item table
item_table_sort_df = item_table_df.sort_values("Purchase Count", ascending=False)

item_table_sort_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
